In [24]:
from langchain.embeddings import SentenceTransformerEmbeddings
import requests
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
from langchain.schema import Document
from typing import List



In [25]:
class PandasTranformer:
    def __init__(self, df) -> None:
        self.df=df
        
    def load(self)->List[Document]:
        # Iterar sobre las filas del DataFrame y crear documentos con metadatos
        documents = []
        for index, row in self.df.iterrows():
            document = {
                'content': row['UEN',:],  # Contenido del documento
                'metadata': {
                    'fecha': row['Fecha'],     # Metadato de fecha
                    # Agrega más metadatos aquí si es necesario
                }
            }
            documents.append(document)

        # Ahora la lista 'documents' contiene documentos con sus metadatos
        print(documents)
        return documents






In [26]:
## CARGA DE PRESUPUESTOS
sheet_id='1ofJ38O2-dLfw7TZOxeTDYUzlWgl6hN2iMLZCMscTgGo'
hoja='Presupuesto'
gsheet_url_presupuesto = "https://docs.google.com/spreadsheets/d/{}/gviz/tq?tqx=out:csv&sheet={}".format(sheet_id, hoja)
df_presupuesto=pd.read_csv(gsheet_url_presupuesto)
df_presupuesto = df_presupuesto.convert_dtypes()
df_presupuesto['FECHASQL'] = pd.to_datetime(df_presupuesto['FECHASQL'], format='%d/%m/%Y')
df_presupuesto=df_presupuesto.dropna(axis=1)


#Quiero hacer un dataframe nuevo


In [40]:

urls = [gsheet_url_presupuesto]

ml_papers = []

df_presupuesto["FECHASQL"] = pd.to_datetime(df_presupuesto["FECHASQL"])
df_presupuesto["FECHASQL"] = df_presupuesto["FECHASQL"].dt.strftime("%Y-%m-%d")

loader = CSVLoader('C:/Users/gmartinez/Desktop/new/ProyectoChatBot/archivos/presupuesto_data.csv')
data = loader.load()
ml_papers.append(data)

text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        length_function=len,
        chunk_overlap=50
)



In [38]:

print(documents[200])
type(documents[0].page_content)

page_content=': 200\nFECHASQL: 2023-05-15\nUEN: MERC GUAYMALLEN\nDia: lunes\nPorcentaje: 3.54%\nCODPRODUCTO: GNC\nVENTAS: 3575.4' metadata={'source': 'C:/Users/gmartinez/Desktop/new/ProyectoChatBot/archivos/presupuesto_data.csv', 'row': 200}


str

In [39]:
embedding_st= SentenceTransformerEmbeddings(
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

def convert_documents_to_strings(documents):
    for document in documents:
        document = document.text.replace("\n", " ")

documents = convert_documents_to_strings(documents)


#inscrustaciones= embedding_st.embed_documents(documents)

AttributeError: 'Document' object has no attribute 'text'

In [18]:
!pip install sentence_transformers

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.2/1.2 MB 715.9 kB/s eta 0:00:00
     -------------------------------------- 977.5/977.5 kB 1.9 MB/s eta 0:00:00
     -------------------------------------- 172.3/172.3 MB 2.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125960 sha256=b5a92d9b1e03473719e5ee3d71fb2bdf270fec3c9754c888d0e3a626c9e610a7
  Stored in directory: c:\users\gmartinez\appdata\local\pip\cache\wheels\0a\f5\dd\9d00836c4e9e279c2a59d5b0ab72dafa66cbc626a327c550dd
Successfully built sentence_transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [45]:
len(documents)
print(documents[0])

page_content=': 0\nFECHASQL: 2023-05-01\nUEN: ADOLFO CALLE\nDia: lunes\nPorcentaje: 3.54%\nCODPRODUCTO: GNC\nVENTAS: 2053.2' metadata={'source': 'C:/Users/gmartinez/Desktop/new/ProyectoChatBot/archivos/presupuesto_data.csv', 'row': 0}


In [46]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from rich.console import Console

console = Console()

recreate_chroma_db = False
def get_chroma_db(embeddings, documents, path):

    if recreate_chroma_db:
        return Chroma.from_documents(
            documents=documents, embedding=embeddings, persist_directory=path
        )
    else:
        console.print("CARGANDO CHROMA EXISTENTE")
        return Chroma(persist_directory=path, embedding_function=embeddings)
    
    
vectorstore_chroma = get_chroma_db(embedding_st, data, "chroma")



CARGANDO CHROMA EXISTENTE

In [48]:
print(vectorstore_chroma[0])

TypeError: 'Chroma' object is not subscriptable

In [49]:
vectorstore_chroma.persist()

In [50]:
vectorstore_chroma = Chroma(
    persist_directory= "chroma",
    embedding_function= embedding_st
)

In [51]:
docs = vectorstore_chroma.similarity_search_with_score("Presupuestos del ultimo mes", k=5)

In [52]:
len(docs)
docs[0]

IndexError: list index out of range